In [1]:
import pandas as pd
import requests
from tqdm.notebook import tqdm

In [2]:
PATH_TO_DATASETS = "../Datasets/"

# GREEN SPACES

This dataset describes an overview of public green, except for solitary trees. This data collection includes location, type of vegetation, and management information.

In [3]:
df = pd.read_csv(f'{PATH_TO_DATASETS}openbaar-groen0.csv', sep=';')

In [28]:
df[['LAT', 'LON']] = df['geo_point_2d'].str.split(',', expand=True)
df['LAT'] = pd.to_numeric(df['LAT'], errors='coerce')
df['LON'] = pd.to_numeric(df['LON'], errors='coerce')

In [ ]:
#df.to_csv(f'{PATH_TO_DATASETS}green-spaces.csv', index=False)

In [32]:
df = pd.read_csv(f'{PATH_TO_DATASETS}green-spaces.csv')

In [33]:
df.drop(columns=['geo_point_2d', 'geo_shape', 'TECHN_KWAL', 'STREET'], inplace=True)

In [ ]:
#df.to_csv(f'{PATH_TO_DATASETS}green-spaces.csv', index=False)

# AIR POLLUTION

In [13]:
df = pd.read_csv(f'{PATH_TO_DATASETS}merged_air_pollution_data.csv')

In [15]:
cols_to_convert = ['Lat', 'Lon', 'PM1', 'PM2.5', 'PM10']
for col in cols_to_convert:
    df[col] = df[col].str.replace(',', '.', regex=False)
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [16]:
df['Time_UTC'] = pd.to_datetime(df['Time_UTC'], errors='coerce')
df['Time_Local'] = pd.to_datetime(df['Time_Local'], errors='coerce')

In [ ]:
df.drop(columns=['Time_Local'], inplace=True)